## Content-Based Filtering

In [14]:
import pandas as pd # Data manipulation and analysis
import numpy as np # Numerical operations
import pickle # save and load serialized Python objects
import re # (regular expressions) Text processing and pattern matching
import nltk # (Natural Language Toolkit) Natural language processing
from sklearn.feature_extraction.text import TfidfVectorizer # convert text into numerical form (TF-IDF representation)
from sklearn.metrics.pairwise import cosine_similarity # Measure similarity between text data
from nltk.tokenize import word_tokenize # Split text into individual words

In [15]:
news_df = pd.read_csv('/content/news.csv')
rec_items_df = pd.read_csv('/content/rec_items.csv')
rec_feedback_df = pd.read_csv('/content/rec_feedback.csv')
users_df = pd.read_csv('/content/users.csv')
rec_users_df = pd.read_csv('/content/rec_users.csv')

In [16]:
# Displays the first 5 rows of the news dataset
news_df.head()

,id,title,description,published_date,breaking_news,blob_image,source_url,created_at,updated_at,published_at,created_by_id,updated_by_id,shares,comment_count,type
0,23996,අද ඩොලරයේ අගය,ශ්‍රී ලංකා මහ බැංකුව විසින් අද (13) දින නිකුත්...,2025-02-13 12:10:32.919,False,NaN,https://www.hirunews.lk/396721/%E0%B6%85%E0%B6...,2025-02-13 09:43:07.843,2025-02-13 09:43:12.403,2025-02-13 09:43:12.131,7.0,7.0,0,0,News
1,23995,‘ක්ලීන් ශ‍්‍රී ලංකා ලිඛිතව තියෙනවා.’ මාලිමා මන...,‘ක්‍ලීන් ශ්‍රී ලංකා’ වැඩපිළිවෙල යනු කුමක්දැයි ...,2025-02-13 09:30:00,False,NaN,https://lankacnews.com/%e0%b6%9a%e0%b7%8a%e0%b...,2025-02-13 09:42:37.114,2025-02-13 09:42:47.003,NaN,NaN,NaN,0,0,News
2,23994,පාපන්දු ගෝල කණුවක් කඩා වැටී පාසැල් සිසුවෙකු ජී...,පාසැල් ක්‍රීඩාගංනයක තිබූ පාපන්දු ගෝල කණුවක් කඩ...,2025-02-13 11:10:47.133,False,NaN,https://www.hirunews.lk/396719/%E0%B6%B4%E0%B7...,2025-02-13 09:33:20.185,2025-02-13 09:33:23.796,2025-02-13 09:33:23.747,7.0,7.0,0,0,News
3,23993,සුජීව සේනසිංහගේ මූලික අයිතිවාසිකම් පෙත්සම විභා...,තමන්ට එරෙහිව අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුව විසි...,2025-02-13 10:10:29.598,False,NaN,https://www.hirunews.lk/396716/%E0%B7%83%E0%B7...,2025-02-13 09:31:55.442,2025-02-13 09:31:59.501,2025-02-13 09:31:59.41,7.0,7.0,0,0,News
4,23992,නීති විරෝධී ධීවර දැල් දෙසීය පනහක් නීතියේ රැහැනට,"ශ්‍රී ලංකා නාවික හමුදාව, කිලිනොච්චිය මුද්දලම්ප...",2025-02-13 10:58:56.639,False,NaN,https://www.dinamina.lk/2025/02/13/lawnorder/1...,2025-02-13 09:29:23.368,2025-02-13 09:29:26.299,2025-02-13 09:29:26.258,7.0,7.0,0,0,News


In [17]:
# Displays the data types of each column
news_df.dtypes

,0
id,int64
title,object
description,object
published_date,object
breaking_news,bool
blob_image,float64
source_url,object
created_at,object
updated_at,object
published_at,object


In [18]:
# Prints the number of missing (null) values in each column
print(news_df.isnull().sum())

id                   0
title                0
description          0
published_date       0
breaking_news        0
blob_image        1000
source_url           0
created_at           0
updated_at           0
published_at        35
created_by_id      382
updated_by_id       35
shares               0
comment_count        0
type                 0
dtype: int64


In [19]:
# Removes the specified columns from the DataFrame
news_df.drop(columns=["blob_image", "updated_by_id", "created_by_id", "shares", "comment_count", "type"], inplace=True)

In [20]:
# Fills missing (NaN) values in a column with the most frequently occurring value (mode).

news_df['published_at'] = news_df['published_at'].fillna(news_df['published_at'].mode()[0])

In [21]:
print(news_df.isnull().sum())

id                0
title             0
description       0
published_date    0
breaking_news     0
source_url        0
created_at        0
updated_at        0
published_at      0
dtype: int64


In [22]:
# Helpful for text analysis or natural language processing tasks
news_df['content'] = news_df['title'] + " " + news_df['description']

In [23]:
# Stop words which are don't carry significant meaning

sinhala_stop_words = [
    "අතර", "ඉන්", "එක", "එය", "ඔබ", "ඔයා", "ඔහු", "ඔවුන්",
    "ඕනෑ", "ආදිය", "ඇයි", "ඇතුලත", "ඉන්පසු",
    "ඉස්සර", "එක්", "ඔය", "ඔයාලා", "ඔයාව", "උදාහරණ", "උපුටා",
    "ඉදිරියට","එදා", "ඒ", "ඒවා", "ඕන", "ඔක්කෝම", "ඔවුන්",
    "අප", "අපේ", "ඇතුළත්", "ඇත", "ඊයේ", "ඔබට", "ඔබගේ", "එහි",
    "එන්න","ඕක", "ඉහළ", "ඔබේ", "අය", "ඔව්",
    "අද", "ඉතා", "ඉතාම", "ඉහත", "අවශ්‍ය", "ඔයාලට"
]

In [24]:
def preprocess_sinhala_text(text):
    text = str(text) # Ensure the input is a string
    text = re.sub(r'[^\u0D80-\u0DFF\s]', '', text) # Remove non-Sinhala characters except spaces
    text = text.strip() # Remove leading and trailing spaces
    tokens = word_tokenize(text) # Tokenize the text into words
    tokens = [word for word in tokens if word not in sinhala_stop_words] # Remove stop words
    return " ".join(tokens) # Join tokens back into a cleaned string

In [25]:
# Download tokenizer models
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [27]:
# Apply text preprocessing to each row in the 'content' column
news_df['content'] = news_df['content'].apply(preprocess_sinhala_text)

In [28]:
tfidf_vectorizer = TfidfVectorizer() # Create a TF-IDF vectorizer instance
tfidf_matrix = tfidf_vectorizer.fit_transform(news_df['content']) # Convert the 'content' column into a TF-IDF matrix

In [29]:
# Compute cosine similarity between all text entries
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) # Measures how similar each text entry in news_df['content'] is to every other entry

In [30]:
# Create a DataFrame to store cosine similarity scores with 'id' as index and columns
cosine_sim_df = pd.DataFrame(cosine_sim, index=news_df['id'], columns=news_df['id'])

In [31]:
with open('cbf_model_cosine_sim.pkl', 'wb') as f: # Open a file in write-binary mode to save the cosine similarity model
    pickle.dump(cosine_sim, f) # Serialize and save the cosine similarity matrix to the file

In [32]:
with open('cbf_model_tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)  # Serialize and save the TF-IDF vectorizer

In [33]:
def recommend_content_based(news_id, cosine_sim=cosine_sim, top_n=5):
    if news_id not in news_df['id'].values: # Check if the provided news_id exists in the 'id' column of the DataFrame
        return [] # Return an empty list if the news_id does not exist

    idx = news_df[news_df['id'] == news_id].index[0]  # Get the index of the article with the given news_id
    sim_scores = list(enumerate(cosine_sim[idx])) # Retrieve the cosine similarity scores for the article at the given index
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # Sort the similarity scores in descending order (highest similarity first)
    sim_scores = sim_scores[1:top_n+1] # Get the top N most similar articles (excluding the article itself)
    recommended_indices = [x[0] for x in sim_scores] # Extract the indices of the most similar articles

    return news_df.iloc[recommended_indices]['id'].tolist() # Return the 'id' of the recommended articles as a list

In [34]:
# Call the recommend_content_based function to get the top 5 similar articles to the article with ID 23996
recommended_articles = recommend_content_based(23996, top_n=5)
print(recommended_articles)


[23097, 23794, 23158, 23819, 23710]


## Collaborative Filtering

In [35]:
# For Building recommender systems
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505178 sha256=43b5d8ca48d972e80a1b140e66b92799b997681691f7c5759e0fd88333d8cf7b
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [36]:
import pandas as pd # For handling data in DataFrame format
import numpy as np # For numerical operations and array handling
import pickle # For serializing and saving Python objects
from sklearn.neighbors import NearestNeighbors # For building nearest neighbor models
from surprise import SVD, Dataset, Reader # (Singular Value Decomposition) Dataset/Reader classes from Surprise for collaborative filtering-based recommender systems
from surprise.model_selection import train_test_split # For splitting data into training and test sets
import datetime # For working with dates and times (Helpful for timestamping data or filtering by dates)

In [37]:
rec_feedback_df = pd.read_csv('rec_feedback.csv')

users_df = pd.read_csv('users.csv')

In [38]:
rec_feedback_df.head() # Display the first 5 rows

,id,feedback_type,time_stamp,comment,user_id,item_id,created_at,updated_at,created_by_id,updated_by_id
0,49554,click,2025-02-13 10:03:47.440412,NaN,1182,23942,NaN,NaN,NaN,NaN
1,49553,read,2025-02-13 10:03:42.411734,NaN,1182,23942,NaN,NaN,NaN,NaN
2,49552,click,2025-02-13 10:01:54.947236,NaN,1058,23942,NaN,NaN,NaN,NaN
3,49550,click,2025-02-13 10:01:31.955569,NaN,1058,23930,NaN,NaN,NaN,NaN
4,49549,click,2025-02-13 10:01:10.809916,NaN,1058,23941,NaN,NaN,NaN,NaN


In [39]:
# Display the data types of each column
rec_feedback_df.dtypes

,0
id,int64
feedback_type,object
time_stamp,object
comment,float64
user_id,int64
item_id,int64
created_at,float64
updated_at,float64
created_by_id,float64
updated_by_id,float64


In [40]:
# Print the number of missing (null) values
print(rec_feedback_df.isnull().sum())

id                  0
feedback_type       0
time_stamp          0
comment          1000
user_id             0
item_id             0
created_at       1000
updated_at       1000
created_by_id    1000
updated_by_id    1000
dtype: int64


In [41]:
# Drop unnecessary columns from the DataFrame
rec_feedback_df.drop(columns=["comment", "created_at", "updated_at", "created_by_id", "updated_by_id"], inplace=True)

In [42]:
print(rec_feedback_df.isnull().sum())

id               0
feedback_type    0
time_stamp       0
user_id          0
item_id          0
dtype: int64


In [43]:
interaction_weights = {'click': 1, 'read': 2, 'like': 3} # Define a dictionary mapping interaction types to numerical weights
rec_feedback_df['weight'] = rec_feedback_df['feedback_type'].map(interaction_weights) # Map the interaction type to its corresponding weight and create a new 'weight' column

In [44]:
# Convert 'time_stamp' column to datetime format
rec_feedback_df["time_stamp"] = pd.to_datetime(rec_feedback_df["time_stamp"], errors='coerce')

# Handle missing values if necessary (drop rows with NaN in the 'time_stamp' column)
rec_feedback_df = rec_feedback_df.dropna(subset=["time_stamp"])

In [45]:
from datetime import datetime

current_time = datetime.now() # Get the current date and time as a datetime object
print(current_time)

2025-02-24 03:14:32.815045


In [46]:
current_time = datetime.now() # Get the current date and time
time_decay_factor = 0.9 # Set the time decay factor, which determines how quickly the weight decays over time
rec_feedback_df['time_weight'] = rec_feedback_df['time_stamp'].apply(
    lambda x: time_decay_factor ** ((current_time - x).days)
) # Apply a time decay to each timestamp (Useful for give more importance to recent interactions)

In [47]:
# Calculate the final weight by multiplying the interaction weight with the time decay weight
rec_feedback_df['final_weight'] = rec_feedback_df['weight'] * rec_feedback_df['time_weight']

print(rec_feedback_df[['weight', 'time_weight', 'final_weight']])

     weight  time_weight  final_weight
0       1.0     0.348678      0.348678
1       2.0     0.348678      0.697357
2       1.0     0.348678      0.348678
3       1.0     0.348678      0.348678
4       1.0     0.348678      0.348678
..      ...          ...           ...
995     1.0     0.254187      0.254187
996     1.0     0.254187      0.254187
997     2.0     0.254187      0.508373
998     1.0     0.254187      0.254187
999     1.0     0.254187      0.254187

[1000 rows x 3 columns]


In [48]:
reader = Reader(rating_scale=(0, 5)) # Initialize the Reader object, defining the rating scale from 0 to 5
data = Dataset.load_from_df(rec_feedback_df[['user_id', 'item_id', 'final_weight']], reader) # Load the data from the DataFrame into a Surprise Dataset

In [49]:
trainset, testset = train_test_split(data, test_size=0.2) # Split the data into training and testing sets (80% training, 20% testing)
svd = SVD() # Initialize the SVD model
svd.fit(trainset) # Train the SVD model on the training set

In [50]:
# Save the SVD model to the file
with open('collaborative_model.pkl', 'wb') as f:
    pickle.dump(svd, f)

In [51]:
def recommend_collaborative(user_id, model, rec_feedback_df, top_n=5):
    """Recommend top N articles for a given user using SVD."""

    # Check if the user_id exists in the rec_feedback_df
    if user_id not in rec_feedback_df['user_id'].values:
        print(f"User {user_id} not found. Returning content-based recommendations.")

        return recommend_content_based(news_id=23996, top_n=top_n)

    # Get the unique list of all item IDs from the feedback data
    all_items = rec_feedback_df['item_id'].unique()
    # Generate predictions for the given user and all items
    predictions = [model.predict(user_id, item) for item in all_items] # Predict ratings for each item

    # Sort the predictions by estimated rating (in descending order) and get the top N recommendations
    top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    return [pred.iid for pred in top_recommendations] # Return a list of item IDs for the top N recommended items

In [52]:
user_id = 2329
# Call the recommend_collaborative function to get the top 5 recommended articles for the specified user
top_recommendations = recommend_collaborative(user_id, svd, rec_feedback_df, top_n=5)
print("Top 5 Recommendations:", top_recommendations)

Top 5 Recommendations: [23942, 23930, 23941, 23937, 23953]


## Hybrid Filtering

In [84]:
import random # For generating random numbers or making random selections
import numpy as np # For numerical operations, such as arrays and mathematical functions
import pickle # For serializing and deserializing Python objects
import pandas as pd # For data manipulation and analysis, especially with DataFrames
from surprise import SVD, Dataset, Reader # For building recommendation systems
from sklearn.preprocessing import MinMaxScaler # To scale features to a specified range, typically [0, 1]
from datetime import datetime, timedelta #For handling date and time operations

In [85]:
with open('cbf_model_cosine_sim.pkl', 'rb') as f:
    cosine_sim = pickle.load(f)

with open('cbf_model_tfidf.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('collaborative_model.pkl', 'rb') as f:
    knn = pickle.load(f)

In [86]:
def get_recent_trending_news(rec_feedback_df, top_n=5, days=7):
    """Fetch trending articles based on recent engagement (last 'days')."""

    # Check if the 'timestamp' column exists in the DataFrame
    if "timestamp" not in rec_feedback_df.columns:
        print("No timestamp column found. Falling back to general trending news.")
        return rec_feedback_df["item_id"].value_counts().head(top_n).index.tolist()

    # Convert the 'timestamp' column to datetime format. Invalid timestamps are set to NaT (Not a Time)
    rec_feedback_df["timestamp"] = pd.to_datetime(rec_feedback_df["timestamp"], errors='coerce')

    # Check for invalid timestamps
    if rec_feedback_df["timestamp"].isnull().all():
        print("All timestamps are invalid. Falling back to general trending news.")
        return rec_feedback_df["item_id"].value_counts().head(top_n).index.tolist()

    # Calculate the date for 'days' ago from the current date
    recent_date = datetime.now() - timedelta(days=days)

    # Filter the DataFrame to include only rows where the 'timestamp' is within the last 'days' period
    recent_engagements = rec_feedback_df[rec_feedback_df["timestamp"] >= recent_date]

    if recent_engagements.empty:
        print("No recent engagements found. Falling back to general trending news.")
        return rec_feedback_df["item_id"].value_counts().head(top_n).index.tolist()

    # Get the top N most frequent items from the recent engagements
    trending = (
        recent_engagements["item_id"]
        .value_counts()
        .head(top_n)
        .index.tolist()
    )

    # Return the trending items if found, otherwise return an empty list
    return trending if trending else []

In [87]:
print(rec_feedback_df["time_stamp"].head())

0   2025-02-13 10:03:47.440412
1   2025-02-13 10:03:42.411734
2   2025-02-13 10:01:54.947236
3   2025-02-13 10:01:31.955569
4   2025-02-13 10:01:10.809916
Name: time_stamp, dtype: datetime64[ns]


In [88]:
def calculate_user_alpha(user_id, rec_feedback_df):
    """Dynamically calculate alpha: Higher interactions → Higher CF weight"""

    user_interactions = rec_feedback_df[rec_feedback_df["user_id"] == user_id]
    total_interactions = len(user_interactions)

    if total_interactions == 0:
        return 0.2  # Lower alpha for new users (favor CBF more)

    # Logarithmic scaling for a smoother curve
    alpha = np.log1p(total_interactions) / np.log1p(100)

    return min(1, max(0, alpha))  # Ensure alpha stays in [0,1]

In [89]:
def max_marginal_relevance(recommendations, hybrid_scores, cosine_sim, top_n=5):
    selected_items = []
    for item in recommendations:
        # Penalize items that are too similar to those already selected
        similarity_score = sum(cosine_sim.get(item, {}).get(other_item, 0) for other_item in selected_items)
        adjusted_score = hybrid_scores[item] - similarity_score * 0.7  # Increase the penalty to 0.7
        hybrid_scores[item] = adjusted_score

    # Sort by adjusted hybrid score and return the top N items
    return sorted(hybrid_scores.keys(), key=lambda x: hybrid_scores[x], reverse=True)[:top_n]

In [90]:
# Defines a function to normalize the scores to a range of 0-1 for fair weighting
def normalize_scores(scores):

    if not scores:
        return {} # If there are no scores, returns an empty dictionary

    # Converts the values of the 'scores' dictionary to a NumPy array and reshapes it into a column vector (for scaling)
    values = np.array(list(scores.values())).reshape(-1, 1)

    # Creates an instance of the MinMaxScaler, which scales values to the range [0, 1]
    scaler = MinMaxScaler()

    # Fits the scaler to the values and transforms them to the range [0, 1]
    normalized_values = scaler.fit_transform(values).flatten()

    # Returns a dictionary where each original score is mapped to its normalized value
    return {key: norm_score for key, norm_score in zip(scores.keys(), normalized_values)}

In [91]:
 # Defines a function to get the top N content-based recommendations using cosine similarity
def recommend_content_based(news_id, top_n=5):

    # Checks if the given news_id exists in the cosine_sim dictionary
    if news_id not in cosine_sim:
        return [] # If the news_id is not found, return an empty list

    # Creates a list of tuples, where each tuple contains an index (item ID) and its cosine similarity value to the given news_id
    similar_items = list(enumerate(cosine_sim[news_id]))

    # Sorts the similar items by their cosine similarity value in descending order (most similar first)
    sorted_items = sorted(similar_items, key=lambda x: x[1], reverse=True)

    # Returns a list of the top N most similar item IDs, based on the sorted cosine similarity values
    return [item[0] for item in sorted_items[:top_n]]

In [92]:
# Defines a function to get the top N collaborative filtering recommendations using Singular Value Decomposition (SVD)
def recommend_collaborative(user_id, model, rec_feedback_df, top_n=5):

     # Checks if the user_id exists in the 'user_id' column of the rec_feedback_df DataFrame
    if user_id not in rec_feedback_df['user_id'].values:
        return []

    # Retrieves all unique item IDs from the 'item_id' column in the rec_feedback_df DataFrame
    all_items = rec_feedback_df['item_id'].unique()

    # Uses the SVD to predict the user's rating for each item
    predictions = {item: model.predict(user_id, item).est for item in all_items}

    # Sorts the items based on the predicted rating (est) in descending order (highest predicted rating first)
    # Returns the top N items with the highest predictions
    return sorted(predictions, key=predictions.get, reverse=True)[:top_n]

In [93]:
def recommend_hybrid(user_id, news_id, rec_feedback_df, model=svd, top_n=5):
    """Hybrid recommendation system combining content-based and collaborative filtering."""

    # Filter the DataFrame for user-specific interactions
    user_interactions = rec_feedback_df[rec_feedback_df["user_id"] == user_id]

    # If the user has no interactions, trigger fallback to trending news
    if user_interactions.empty:
        print(f"No data for user {user_id}. Showing recent trending news...")
        return get_recent_trending_news(rec_feedback_df, top_n=top_n) or random.sample(list(rec_feedback_df["item_id"].unique()), top_n)

    # Calculate dynamic alpha based on the user's interaction history
    alpha = calculate_user_alpha(user_id, rec_feedback_df)

    # Fetch content-based recommendations (CBF) for the given news_id
    cbf_recommendations = recommend_content_based(news_id, top_n=top_n)

    # Fetch collaborative filtering recommendations (CF) for the given user
    cf_recommendations = recommend_collaborative(user_id, model, rec_feedback_df, top_n=top_n)

    # Assign cosine similarity scores for the CBF recommendations
    cbf_scores = {item: cosine_sim[news_id][item] for item in cbf_recommendations}

    # Assign predicted scores for the CF recommendations
    cf_scores = {item: model.predict(user_id, item).est for item in cf_recommendations}

    # Normalize both CBF and CF scores to a range of [0, 1]
    cbf_scores = normalize_scores(cbf_scores)
    cf_scores = normalize_scores(cf_scores)

    # Dictionary to store hybrid scores (CBF + CF)
    hybrid_scores = {}

    # Combine the scores from both filtering approaches using the alpha value
    for item in set(cbf_recommendations + cf_recommendations):
        cbf_score = cbf_scores.get(item, 0)  # Default to 0 if no CBF score
        cf_score = cf_scores.get(item, 0)  # Default to 0 if no CF score
        hybrid_scores[item] = alpha * cbf_score + (1 - alpha) * cf_score

    # Apply Maximal Marginal Relevance (MMR) to diversify the recommendations
    top_recommendations = max_marginal_relevance(list(hybrid_scores.keys()), hybrid_scores, cosine_sim, top_n=top_n)

    # If no recommendations were found, trigger fallback to trending news
    if not top_recommendations:
        print(f"No recommendations found for user {user_id}. Showing recent trending news...")
        return get_recent_trending_news(rec_feedback_df, top_n=top_n) or random.sample(list(rec_feedback_df["item_id"].unique()), top_n)

    # Return the top N hybrid recommendations
    return top_recommendations

In [94]:
with open('hybrid_recommendation_model.pkl', 'wb') as f:
    pickle.dump(svd, f)

In [105]:
user_id_1 = 2329
news_id_1 = 23585

recommendations_1 = recommend_hybrid(user_id_1, news_id_1, rec_feedback_df, top_n=5)
print(f"User {user_id_1} Recommendations:", recommendations_1)

User 2329 Recommendations: [23937, 23941, 23942, 23953, 23930]


In [106]:
# New user with no history
user_id_new = 9999
news_id_new = 23996
recommendations_new = recommend_hybrid(user_id_new, news_id_new, rec_feedback_df, top_n=5)
print(f"User {user_id_new} (no history) - Fallback Recommendations:", recommendations_new)

No data for user 9999. Showing recent trending news...
No timestamp column found. Falling back to general trending news.
User 9999 (no history) - Fallback Recommendations: [23749, 23769, 23820, 23930, 23786]
